# Databases, Iterators &amp; Looping #

This notebook will talk about how I do looping over database queries.

The database code will use the [APSW](https://rogerbinns.github.io/apsw/) wrapper around the [SQLite](https://sqlite.org/index.html) DBMS.

Make sure these are installed before executing the following imports:

In [ ]:
import sys
import os
import apsw as sqlite


Create the temporary example database, using data extracted from [Wikipedia](https://en.wikipedia.org/wiki/Natural_satellite#Natural_satellites_of_the_Solar_System):

In [ ]:
db = sqlite.Connection \
  (
    ":memory:",
    flags = sqlite.SQLITE_OPEN_READWRITE | sqlite.SQLITE_OPEN_CREATE
  )

planets = \
    [
        (1, "Mercury"),
        (2, "Venus"),
        (3, "Terra"),
        (4, "Mars"),
        (5, "Jupiter"),
        (6, "Saturn"),
        (7, "Uranus"),
        (8, "Neptune"),
    ]
moons = \
    [ # only a partial list!
        (1, "Luna", 3),
        (2, "Deimos", 4),
        (3, "Phobos", 4),
        (4, "Callisto", 5),
        (5, "Europa", 5),
        (6, "Ganymede", 5),
        (7, "Io", 5),
        (8, "Dione", 6),
        (9, "Hyperion", 6),
        (10, "Iapetus", 6),
        (11, "Mimas", 6),
        (12, "Enceladus", 6),
        (13, "Rhea", 6),
        (14, "Tethys", 6),
        (15, "Titan", 6),
        (16, "Ariel", 7),
        (17, "Miranda", 7),
        (18, "Oberon", 7),
        (19, "Titania", 7),
        (20, "Umbriel", 7),
        (21, "Nereid", 8),
        (22, "Triton", 8),
    ]

cu = db.cursor()
cu.execute \
  (
    "create table planets\n"
    "  (\n"
    "    id integer primary key,\n"
    "    name text\n"
    "  )\n"
  )
cu.execute \
  (
    "create table moons\n"
    "  (\n"
    "    id integer primary key,\n"
    "    name text,\n"
    "    parent integer\n"
    "  )\n"
  )
for item in planets :
    cu.execute("insert into planets(id, name) values (?, ?)", item)
#end for
for item in moons :
    cu.execute \
      (
        "insert into moons(id, name, parent) values (?, ?, ?)",
        item
      )
#end for
cu.close()
del cu

For convenient iteration over query results, I like to use the following function:

In [ ]:
def db_iter(conn, cmd, mapfn = lambda x : x) :
    "executes cmd on a new cursor from connection conn and" \
    " yields the results in turn."
    cu = conn.cursor()
    result = cu.execute(cmd)
    while True :
        try :
            yield mapfn(next(result))
        except StopIteration :
            break
        #end try
    #end while
#end db_iter


This wraps up the common tasks of creating a cursor, executing a query, and then retrieving the results. It also has the option of putting each result tuple through a mapping function; we will see the usefulness of this in a moment.

For a simple example, here’s a list of all the planets in alphabetical order:

In [ ]:
list(db_iter(db, "select name from planets order by name"))

Since in this example each record only contains a single field, let’s use the `mapfn` option to strip the field value out of the result tuples:

In [ ]:
list(db_iter
  (
    db,
    "select name from planets order by name",
    mapfn = lambda x : x[0]
  ))

It is easy to build other convenience routines on top of the `db_iter` workhorse. For example, another common task is retrieving counts of records:

In [ ]:
def db_count(db, criterion) :
    return \
        next(db_iter(db, "select count(*) from " + criterion))[0]
#end db_count

For instance, how many moons have “ion” in their name?

In [ ]:
db_count(db, "moons where name like '%ion%'")

Now, let’s do a complete listing of all the planets and their moons. Here’s an initial attempt, outputting a nicely HTML-formatted table. Note how the query uses a `left join`; what would happen if we used, say, an `inner join`?

In [ ]:
import io
import html
from IPython.display import \
    HTML

In [ ]:
out = io.StringIO()
out.write("<TABLE>\n")
out.write("<TR><TH>Planet</TH><TH>Moon</TH></TR>\n")
for item in db_iter \
  (
    db,
    "select planets.name, moons.name"
    " from planets left join moons on planets.id = moons.parent"
    " order by planets.name, moons.name"
  ) \
:
    out.write("<TR><TD>")
    out.write(html.escape(item[0]))
    out.write("</TD><TD>")
    if item[1] != None :
        out.write(html.escape(item[1]))
    else :
        out.write("<I>(none)</I>")
    #end if
    out.write("</TD></TR>\n")
#end for
out.write("</TABLE>\n")
HTML(out.getvalue())

That has all the info, but it can be presented better. To start with, let’s only output the planet name for its first entry line. How can we do that? Also, just to complicate things a little, let’s add a count line after the last moon for each planet, showing how many moons there were.

The basic outline of the reporting loop will become like this:

    create database query iterator
    loop
        get next item from iterator
        if item is None or planet has changed :
            output count line for current planet
            indicate a change to current planet
        if item is None :
            terminate loop
        output first/next line for current planet,
            incrementing count as appropriate
    end loop

Here is the actual code:

In [ ]:
out = io.StringIO()
out.write("<TABLE>\n")
out.write("<TR><TH>Planet</TH><TH>Moon</TH></TR>\n")
items_iter = db_iter \
  (
    db,
    "select planets.name, moons.name"
    " from planets left join moons on planets.id = moons.parent"
    " order by planets.name, moons.name"
  )
cur_planet = None
while True :
    item = next(items_iter, None)
    if item == None or cur_planet != None and item[0] != cur_planet :
        if cur_planet != None :
            out.write \
              (
                "<TR><TD><I>Count:</I></TD><TD>%d</TD></TR>\n" % count
              )
            cur_planet = None
        #end if
        if item == None :
            break
    #end if
    out.write("<TR><TD>")
    if cur_planet == None :
        # first moon for next planet
        cur_planet = item[0]
        count = 0
        out.write("<B>" + html.escape(item[0]) + "</B>")
    else :
        # second or subsequent moon, don’t repeat planet name
        out.write("&nbsp;")
    #end if
    out.write("</TD><TD>")
    if item[1] != None :
        # there is a moon
        out.write(html.escape(item[1]))
        count += 1
    else :
        # that’s no moon
        out.write("<I>(none)</I>")
    #end if
    out.write("</TD></TR>\n")
#end while
out.write("</TABLE>\n")
HTML(out.getvalue())


This kind of reporting across two tables, where entries in a child table are grouped according to matching values for a field in the parent table, has various names, such as a *level break* or *break field*. The same basic loop structure can be used in a lot of these cases.